In [ ]:
################################################
# New style 
################################################
import sys
import os

workdir_ = '/glade/work/juliob'
if ( workdir_ not in sys.path ):
    sys.path.append(workdir_)
    print( f" a path to {workdir_} added in {__name__} ")

from myPythonTools.Utils import utils as uti
from myPythonTools.Utils import numerical_utils as nuti
from myPythonTools.Utils import AveragingUtils as Av
from myPythonTools.Utils import validation_data as Val
from myPythonTools.Utils import PlotUtil as Pu
from myPythonTools.Plotting import LatLonMaps as LL 
from myPythonTools.CASutils import filter_utils as fu 

from PyRegridding.Utils import GridUtils as GrU
from PyRegridding.Utils import MakePressures as MkP
from PyRegridding.Drivers import RegridField as RgF


# The usual
from datetime import date
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.tri as tri

# Cartopy for pretty maps
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Some other useful packages 
import importlib
import copy
import time
import cftime

importlib.reload( uti )
importlib.reload( Pu )
importlib.reload(Av)
importlib.reload(Val)
#importlib.reload(vAB)
importlib.reload(MkP)
importlib.reload(RgF)


############################
# Initialize dict for regridding
############################
RgObs={}

In [ ]:
Din='/glade/campaign/cgd/amp/juliob/ERA5/fv1x1_monthly/ZYX/'
Dex='/glade/campaign/cgd/amp/juliob/ERA5/fv1x1_monthly_climo/ZYX/'

for mo in np.arange( start=1, stop=13 ):
    Fin=f'{Din}/ERA5.fv1x1_zERA.time.*-{str(mo).zfill(2)}.nc'
    Fex=f'{Dex}/ERA5.fv1x1_zERA.time.1979_2022-{str(mo).zfill(2)}.nc'
    
    Xin = xr.open_mfdataset( Fin )
    
    # Average along time, keeping the time dim as size-1
    Xin_mean = Xin.mean(dim='time', keep_attrs=True).expand_dims(time=[Xin.time.values[0]])
    
    
    ##########################
    # Add time bounds
    ##########################
    
    # Define the bounds
    bnds = np.array([
        np.datetime64(f"1979-{str(mo).zfill(2)}-01T00:00:00",'ns'),
        np.datetime64(f"2023-{str(mo).zfill(2)}-01T00:00:00",'ns')
    ])
    
    # Expand to match dimensions (1 time, 2 bounds)
    time_bnds = xr.DataArray(
        bnds[np.newaxis, :],  # shape (1, 2)
        dims=("time", "nbnd"),
        coords={"time": Xin_mean.time, "nbnd": [0, 1]},
        name="time_bnds"
    )
    
    # Assign to the Dataset
    Xin_mean["time_bnds"] = time_bnds
    
    
    Xin_mean.to_netcdf( Fex )
    print(f"Wrote {Fex} ")